In [ ]:
import pandas as pd
import sqlalchemy
import numpy as np
import seaborn as sns
from sqlalchemy.exc import SQLAlchemyError
from sklearn.preprocessing import StandardScaler

from db_queries import username, password, dsn, dbhostname, service_name, dbtables, querys
from table_functions import *
from analyze_visualisation import *

In [ ]:
data = {}

try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

data = drop_unused_columns(data)

In [ ]:
final_table = combine_final_table(data)
final_table = create_final_status(final_table)
final_table = normalize_data(final_table)

In [ ]:
pd.set_option('display.max_columns', None)
final_table.head(3)

In [ ]:
#data anal
status_1_data = final_table[final_table['our_final_status'] == 1].sample(n=200, random_state= 69)
status_2_data = final_table[final_table['our_final_status'] == 2].sample(n=200, random_state= 69)
random_to_analyze = pd.concat([status_1_data, status_2_data], ignore_index=True)

In [ ]:
print(list(random_to_analyze.keys()))

In [ ]:
col_to_analyze = ['czas_fazy_1', 'czas_fazy_2', 'czas_fazy_3', 'max_predkosc', 'cisnienie_tloka', 'cisnienie_koncowe','nachdruck_hub', 
                  'anguss', 'temp_pieca', 'oni_temp_curr_f1', 'oni_temp_curr_f2', 'oni_temp_fore_f1', 'oni_temp_fore_f2', 'vds_air_pressure',
                    'vds_vac_hose1', 'vds_vac_hose2', 'vds_vac_tank', 'vds_vac_valve1', 'vds_vac_valve2', 'czas_taktu']
col_flow = ['flow_1', 'flow_2', 'flow_3', 'flow_4', 'flow_5', 'flow_6', 'flow_7', 'flow_8', 'flow_9', 'flow_10', 'flow_11', 'flow_12', 'flow_13',
             'flow_14', 'flow_15', 'flow_16', 'flow_17', 'flow_18', 'flow_19', 'flow_20', 'flow_21', 'flow_22', 'flow_23', 'flow_24', 'flow_25', 'flow_26', 'flow_27', 'flow_28']

#make_and_save_pariplot(random_to_analyze, col_to_analyze, 'normalized_dirst_20.png')
make_and_save_pariplot(random_to_analyze, col_flow, 'normalized_flow.png')